In [40]:
import logging
import re
import os
import gensim
from gensim import corpora, models, similarities
from random import sample

logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO 

In [41]:
CAMELCASE_TO_UNDERSCORE_RE = re.compile('((?<=[a-z0-9])[A-Z]|(?!^)[A-Z](?=[a-z]))')
dictionary = corpora.dictionary.Dictionary()


class CodeWalker(object):
    
    stop_words = ['self', 'the', 'if', 'def', 'in', 'name', 'return', 'to', 'for', 'of', 'import', 'is', 'from', 'none', 'class', 'get', 'and', 'not', 'path', 'bhadron', 'mdst', 'data', 'id', 'set', 'test', 'or', 'assert', 'file', 'error', 'type', 'value', 'true', 'list', 'os', 'this', 'text', 'else', 'with', 'u0627', 'key', 'as', 'cb', 'url', 'lh', 'lhcb', 'lfn', 'collision12', '00020198', 'args', 'false', 'be', 'field', 'format', 'user', 'request', 'add', 'string', 'version', 'print', 'node', 'str', 'object', 'it', 'info', 'default', 'line', 'append', 'by', 'len', 'start', 'message', 'all', 'input', 'license', 'response', 'init', 'config', 'that', 'raise', 'code', '0000', 'try', 'model', 'index', 'size', 'each', 'on', 'an', 'except', 'html', 'u064a', 'dict', 'pass', 'obj', 'result', 'base', 'http', 'elif', 'end', 'join', 'output', 'ref', 'param', 'number', 'new', 'options', 'log', 'server', 'content', 'xsc', 'time', 'feature', 'no', 'equal', 'use', 'create', 'sys', 'u0644', 'db', 'image', 'out', 'int', 'exception', 'kwargs', 'python', 'you', 'any', 'item', 'u0646', 'parser', 'method', 'tree', 'project', 'are', 'token', 'parameters', 'write', 'open', 'group', 'u0648', 'count', 'u0430', 'attr', 'values', 'dir', 'option', 'root', 'segment', 'client', 'source', 'will', 'json', 'element', 'filter', 'debug', 'returns', 'read', 'files', 'max', 'current', 'can', 'context', 'attrs', 'page', 'tag', 'action', 'template', 'module', 'table', 'state', 'status', '66', 'np', 'port', 'function', 'django', 'command', 'parse', 'u0631', 're', 'title', 'check', 'update', 'xmlns', 'uffff', 'software', 'session', 'password', 'at', 'params', 'app', 'main', 'models', 'run', 'date', 'match', 'stream', 'py', 'range', 'has']
    
    
    def convert_camel_case_to_underscore(self, word):
        return CAMELCASE_TO_UNDERSCORE_RE.sub(r'_\1', word).lower()

    def clean_and_split_codewords(self, word):
        """
        HTTPResonse => http, response
        aNewWorld => a, new, world
        a_new_world => a, new, world

        """
        for subword in self.convert_camel_case_to_underscore(word).split('_'):
            if subword and len(subword) > 1 \
              and subword not in self.stop_words:
                yield subword
    

    def __iter__(self):
        for root, dirs, files in os.walk('/Users/camerondavidson-pilon/code/PyconCanada2015/scrapers/data'):
            for file in files:
                if file.endswith('.py'):
                    with open(os.path.join(root, file), 'r') as open_file:
                        lines = open_file.read()
                        for raw_word in re.findall('(\w+)', lines):
                            for clean_words in self.clean_and_split_codewords(raw_word):
                                yield clean_words


    def iter_dictionaries(self):
        for root, dirs, files in os.walk('/Users/camerondavidson-pilon/code/PyconCanada2015/scrapers/data'):
            for file in files:
                if file.endswith('.py'):
                    with open(os.path.join(root, file), 'r') as open_file:
                        lines = open_file.read()
                        result = []
                        for raw_word in re.findall('(\w+)', lines):
                            for clean_words in self.clean_and_split_codewords(raw_word):
                                result.append(clean_words)
                    yield dictionary.doc2bow(result, allow_update=True)

In [42]:
from collections import Counter
c = Counter()
cw = CodeWalker()
for word in cw.iter_subsample_lines():
    c.update([word])
    


TypeError: expected string or buffer

In [43]:
%debug

> /Users/camerondavidson-pilon/.virtualenvs/data/lib/python2.7/re.py(181)findall()
    180     Empty matches are included in the result."""
--> 181     return _compile(pattern, flags).findall(string)
    182 

ipdb> u
> <ipython-input-41-82dc2d04c602>(45)iter_subsample_lines()
     44                             for clean_words in self.clean_and_split_codewords(raw_word):
---> 45                                 yield clean_words
     46 

ipdb> lines
['t', '\n', 's', 'o', ' ', ':', 'c', 't', 't', 'm', 'p', '\n', 'r', 'f', 's', 'p', 'e', ' ', '_', 'A', 'd', 'r', 'n', 'g', '8', 'i', 'd', 'i', 'g', ' ', 't', 'e', ' ', 's', ' ', ' ', ' ', '=', 't', ' ', ',', 'l', ' ', 's', ' ', 'r', '_', '/', 'r', 'i']
ipdb> open_file.read()
*** ValueError: I/O operation on closed file
ipdb> open(os.path.join(root, file), 'r').read_lines()
*** AttributeError: 'file' object has no attribute 'read_lines'
ipdb> open(os.path.join(root, file), 'r').read()
'# -*- coding: utf-8 -*-\nimport urllib2\nimport xml.etr

AttributeError: 'NoneType' object has no attribute 'set_completer_delims'

In [30]:
print map(lambda v: v[0], c.most_common(200))

['self', 'the', 'if', 'def', 'in', 'name', 'return', 'to', 'for', 'of', 'import', 'is', 'from', 'none', 'class', 'get', 'and', 'not', 'path', 'bhadron', 'mdst', 'data', 'id', 'set', 'test', 'or', 'assert', 'file', 'error', 'type', 'value', 'true', 'list', 'os', 'this', 'text', 'else', 'with', 'u0627', 'key', 'as', 'cb', 'url', 'lh', 'lhcb', 'lfn', 'collision12', '00020198', 'args', 'false', 'be', 'field', 'format', 'user', 'request', 'add', 'string', 'version', 'print', 'node', 'str', 'object', 'it', 'info', 'default', 'line', 'append', 'by', 'len', 'start', 'message', 'all', 'input', 'license', 'response', 'init', 'config', 'that', 'raise', 'code', '0000', 'try', 'model', 'index', 'size', 'each', 'on', 'an', 'except', 'html', 'u064a', 'dict', 'pass', 'obj', 'result', 'base', 'http', 'elif', 'end', 'join', 'output', 'ref', 'param', 'number', 'new', 'options', 'log', 'server', 'content', 'xsc', 'time', 'feature', 'no', 'equal', 'use', 'create', 'sys', 'u0644', 'db', 'image', 'out', 'int

In [28]:
len(c)

64013

Counter({'self': 77211, 'if': 21576, 'the': 20873, 'def': 20308, 'in': 18028, 'return': 15003, 'name': 14379, 'class': 13194, 'for': 13044, 'to': 12523, 'none': 11437, 'is': 10569, 'import': 10182, 'of': 10163, 'set': 9837, 'from': 9713, 'data': 8741, 'and': 8581, 'get': 8574, 'not': 8249, 'id': 8082, 'file': 7364, 'or': 7284, 'test': 7282, 'path': 7269, 'assert': 6837, 'error': 6403, 'true': 6147, 'value': 5971, 'type': 5910, 'text': 5596, 'else': 5140, 'os': 5132, 'list': 4913, 'line': 4823, 'input': 4813, 'each': 4812, 'with': 4659, 'xsc': 4574, 'node': 4552, 'url': 4456, 'print': 4393, 'feature': 4338, 'false': 4290, 'this': 4158, 'be': 3969, 'as': 3884, 'key': 3826, 'format': 3798, 'pass': 3738, 'add': 3686, 'start': 3565, 'segment': 3553, 'str': 3553, 'user': 3420, 'obj': 3413, 'args': 3385, 'append': 3336, 'len': 3278, 'index': 3266, '66': 3254, 'end': 3236, 'parameters': 3236, 'code': 3208, 'token': 3203, 'object': 3165, 'uffff': 3131, 'version': 3097, 'string': 3054, 'raise': 

In [32]:
cw = CodeWalker()
corpora.MmCorpus.serialize('../scrapers/data/test_corpus.mm', cw.iter_dictionaries())

INFO:gensim.corpora.mmcorpus:storing corpus in Matrix Market format to ../scrapers/data/test_corpus.mm
INFO:gensim.matutils:saving sparse matrix to ../scrapers/data/test_corpus.mm
INFO:gensim.matutils:PROGRESS: saving document #0
INFO:gensim.matutils:PROGRESS: saving document #1000
INFO:gensim.matutils:PROGRESS: saving document #2000
INFO:gensim.matutils:PROGRESS: saving document #3000
INFO:gensim.matutils:PROGRESS: saving document #4000
INFO:gensim.matutils:PROGRESS: saving document #5000
INFO:gensim.matutils:saved 5404x63813 matrix, density=0.174% (600580/344845452)
INFO:gensim.corpora.indexedcorpus:saving MmCorpus index to ../scrapers/data/test_corpus.mm.index


In [33]:
mm = corpora.MmCorpus('../scrapers/data/test_corpus.mm')

INFO:gensim.corpora.indexedcorpus:loaded corpus index from ../scrapers/data/test_corpus.mm.index
INFO:gensim.matutils:initializing corpus reader from ../scrapers/data/test_corpus.mm
INFO:gensim.matutils:accepted corpus with 5404 documents, 63813 features, 600580 non-zero entries


In [34]:
print mm

MmCorpus(5404 documents, 63813 features, 600580 non-zero entries)


In [35]:
lda = gensim.models.ldamodel.LdaModel(corpus=mm, id2word=dictionary, num_topics=20, update_every=0, passes=10)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.05
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running batch LDA training, 20 topics, 10 passes over the supplied corpus of 5404 documents, updating model once every 5404 documents, evaluating perplexity every 5404 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #2000/5404
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #4000/5404
INFO:gensim.models.ldamodel:-12.852 per-word bound, 7394.9 perplexity estimate based on a held-out corpus of 1404 documents with 457788 words
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #5404/5404
INFO:gensim.models.ldamodel:topic #0 (0.050): 0.003*settings + 0.003*column + 0.002*mock + 0.002*child + 0.002*res + 0.002*items + 0.002*10 + 0.002*other + 0.002*yield + 0.002*property
INFO:gensim.models.ldamodel:topic #9 (0.050): 0.004*settings + 0.003*length + 0.

In [37]:
for t in lda.show_topics():
    print t
    print 

0.008*task + 0.006*service + 0.006*issue + 0.006*argument + 0.006*call + 0.005*queue + 0.005*pool + 0.005*isinstance + 0.005*help + 0.005*func

0.024*descriptor + 0.019*proto + 0.014*extension + 0.013*unittest + 0.011*pb2 + 0.011*repeated + 0.010*enum + 0.008*nested + 0.008*fields + 0.008*extensions

0.030*u0628 + 0.029*u0633 + 0.028*u0645 + 0.026*u062a + 0.023*0006230529595015577 + 0.022*u06cc + 0.020*u062f + 0.019*u0629 + 0.014*u0643 + 0.013*u063a

0.008*xml + 0.007*description + 0.007*document + 0.007*setup + 0.006*language + 0.006*author + 0.006*packages + 0.005*doc + 0.005*convert + 0.005*elements

0.039*x20 + 0.017*x65 + 0.013*x74 + 0.013*testinfo + 0.012*x69 + 0.011*items + 0.010*x73 + 0.010*visit + 0.010*x61 + 0.010*x6f

0.014*u0438 + 0.013*helix + 0.012*u043d + 0.011*stack + 0.010*u0440 + 0.009*u0441 + 0.009*u043e + 0.009*u0435 + 0.008*u043a + 0.007*spring

0.007*entry + 0.007*column + 0.005*hash + 0.005*mock + 0.005*msgid + 0.005*pedido + 0.004*other + 0.004*stdout + 0.004*sp